# 搭建循环神经网络
递归神经网络（RNN）对于自然语言处理和其他序列任务非常有效，因为它们具有“记忆”功能。 它们可以一次读取一个输入$x^{\langle t \rangle}$ 
 （如单词），并且通过隐藏层激活从一个时间步传递到下一个时间步来记住一些信息/上下文，这允许单向RNN从过去获取信息来处理后面的输入，双向RNN可以从过去和未来中获取上下文。

有些东西需要声明：

- 上标$[l]$ 表示第$l$ 层

  - 举例：$a^{[4]}$表示第4层的激活值，$W^{[5]}$ 与$b^{[5]}$ 是第5 层的参数。
- 上标$(i)$表示第$(i)$个样本

  - 举例：$x^{(i)}$表示第i个输入的样本。
- 上标$\langle t \rangle$ 表示第t个时间步

  - 举例：$x^{\langle t \rangle}$表示输入x 的第t 个时间步，$x^{(i)\langle t \rangle}$ 表示输入x 的第i个样本的第t个时间步。
- 下标i表示向量的第i项

  - 举例：$a^{[l]}_i$表示l层中的第i个项的激活值。我们先来加载所需要的库：

In [1]:
import numpy as np
import rnn_utils

## 循环神经网络的前向传播
 我们来看一下下面的循环神经网络的图，在这里使用的是$T_x = T_y$. ，我们来实现它。
 <img src="images/1.png" style="width:500;height:300px;">
我们怎么才能实现它呢？有以下步骤：
- 实现RNN的一个时间步所需要计算的东西。
- 在$T_x$时间步上实现一个循环，以便一次处理所有输入。

### RNN单元
循环神经网络可以看作是单元的重复，首先要实现单个时间步的计算，下图描述了RNN单元的单个时间步的操作
<img src="images/2.png" style="width:500;height:300px;">
现在我们要根据图2来实现一个RNN单元，这需要由以下几步完成：

- 使用tanh函数计算隐藏单元的激活值： $a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a)$.
- 使用 $a^{\langle t \rangle}$ 计算$\hat{y}^{\langle t \rangle} = softmax(W_{ya} a^{\langle t \rangle} + b_y)$.softmax在rnn_utils内。

- 把$(a^{\langle t \rangle}, a^{\langle t-1 \rangle}, x^{\langle t \rangle}, parameters)$存储到cache中。

- 返回 $a^{\langle t \rangle}$ , $y^{\langle t \rangle}$ 与cache。

我们将向量化$m$个样本，因此， $x^{\langle t \rangle}$ 的维度为 $(n_x,m)$.$a^{\langle t \rangle}$
 的维度为 $(n_a,m)$. 

In [2]:
def rnn_cell_forward(xt, a_prev, parameters):
    """
    根据图2实现RNN单元的单步前向传播
    
    参数：
        xt -- 时间步“t”输入的数据，维度为（n_x, m）
        a_prev -- 时间步“t - 1”的隐藏隐藏状态，维度为（n_a, m）
        parameters -- 字典，包含了以下内容:
                        Wax -- 矩阵，输入乘以权重，维度为（n_a, n_x）
                        Waa -- 矩阵，隐藏状态乘以权重，维度为（n_a, n_a）
                        Wya -- 矩阵，隐藏状态与输出相关的权重矩阵，维度为（n_y, n_a）
                        ba  -- 偏置，维度为（n_a, 1）
                        by  -- 偏置，隐藏状态与输出相关的偏置，维度为（n_y, 1）
    
    返回：
        a_next -- 下一个隐藏状态，维度为（n_a， m）
        yt_pred -- 在时间步“t”的预测，维度为（n_y， m）
        cache -- 反向传播需要的元组，包含了(a_next, a_prev, xt, parameters)
    """
    
    # 从“parameters”获取参数
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    # 使用上面的公式计算下一个激活值
    a_next = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, xt) + ba)
    
    # 使用上面的公式计算当前单元的输出
    yt_pred = rnn_utils.softmax(np.dot(Wya, a_next) + by)
    
    # 保存反向传播需要的值
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

### RNN的前向传播
可以看到的是RNN是刚刚构建的单元格的重复连接，如果输入的数据序列经过10个时间步，那么将复制RNN单元10次，每个单元将前一个单元中的隐($a^{\langle t-1 \rangle}$)和当前时间步的输入数据 ($x^{\langle t \rangle}$)作为输入。 它为此时间步输出隐藏状态($a^{\langle t \rangle}$)和预测($y^{\langle t \rangle}$)
<img src="images/3.png" style="width:800px;height:300px;">

In [3]:
def rnn_forward(x, a0, parameters):
    """
    根据图3来实现循环神经网络的前向传播
    
    参数：
        x -- 输入的全部数据，维度为(n_x, m, T_x)
        a0 -- 初始化隐藏状态，维度为 (n_a, m)
        parameters -- 字典，包含了以下内容:
                        Wax -- 矩阵，输入乘以权重，维度为（n_a, n_x）
                        Waa -- 矩阵，隐藏状态乘以权重，维度为（n_a, n_a）
                        Wya -- 矩阵，隐藏状态与输出相关的权重矩阵，维度为（n_y, n_a）
                        ba  -- 偏置，维度为（n_a, 1）
                        by  -- 偏置，隐藏状态与输出相关的偏置，维度为（n_y, 1）
    
    返回：
        a -- 所有时间步的隐藏状态，维度为(n_a, m, T_x)
        y_pred -- 所有时间步的预测，维度为(n_y, m, T_x)
        caches -- 为反向传播的保存的元组，维度为（【列表类型】cache, x)）
    """
    
    # 初始化“caches”，它将以列表类型包含所有的cache
    caches = []
    
    # 获取 x 与 Wya 的维度信息
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    # 使用0来初始化“a” 与“y”
    a = np.zeros([n_a, m, T_x])
    y_pred = np.zeros([n_y, m, T_x])
    
    # 初始化“next”
    a_next = a0
    
    # 遍历所有时间步
    for t in range(T_x):
        ## 1.使用rnn_cell_forward函数来更新“next”隐藏状态与cache。
        a_next, yt_pred, cache = rnn_cell_forward(x[:, :, t], a_next, parameters)
        
        ## 2.使用 a 来保存“next”隐藏状态（第 t ）个位置。
        a[:, :, t] = a_next
        
        ## 3.使用 y 来保存预测值。
        y_pred[:, :, t] = yt_pred
        
        ## 4.把cache保存到“caches”列表中。
        caches.append(cache)
    
    # 保存反向传播所需要的参数
    caches = (caches, x)
    
    return a, y_pred, caches

我们构建了循环神经网络的前向传播函数，这对于某些应用程序来说已经足够好了，但是它还存在梯度消失的问题。当每个输出$y^{\langle t \rangle}$ 是根据局部的上下文来进行预测的时候，它的效果是比较好的（意思是输入的是$x^{\langle t' \rangle}$ ,其中 $t'$ 与$t$相隔不是太远）。

 接下来我们要构建一个更加复杂的LSTM模型，它可以更好地解决梯度消失的问题，LSTM能够更好地记住一条信息，并且可以在很多时间步中保存。

## 长短时记忆 (LSTM)网络
<img src="images/4.png" style="width:500;height:400px;">

### “门”的介绍
#### 遗忘门
假设我们正在阅读文本中的单词，并希望使用LSTM来跟踪语法结构，比如主语是单数还是复数。如果主语从单数变为复数，我们需要找到一种方法来摆脱我们先前存储的单复数状态的记忆值。在LSTM中，遗忘门是这样做的:
$$\Gamma_f^{\langle t \rangle} = \sigma(W_f[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_f)\tag{1} $$
其中，$W_f$是控制遗忘门的权值，我们把 $a^{\langle t-1 \rangle}$, $x^{\langle t \rangle}$ 连接起来变为$[a^{\langle t-1 \rangle}, x^{\langle t \rangle}]$ ,然后乘以 $W_f$，结果就是得到了一个矢量 $\Gamma_f^{\langle t \rangle}$,其值在0与1 之间。这个遗忘门向量将与前一个单元状态 $c^{\langle t-1 \rangle}$ 相乘，因此，如果 $\Gamma_f^{\langle t \rangle}$ 的一个值是0 （或者≈0），则意味着LSTM应该删除对应的信息，如果其中有为1 的值，那么LSTM将保留该信息。
#### 更新门
一旦我们“忘记”所讨论的过去的主题是单数，我们需要找到一种方法来更新它，以反映新的主题现在是复数。这里是更新门的公式：
$$\Gamma_u^{\langle t \rangle} = \sigma(W_u[a^{\langle t-1 \rangle}, x^{\{t\}}] + b_u)\tag{2} $$ 
与遗忘门相似， $\Gamma_u^{\langle t \rangle}$ 向量的值是在0 与1 之间，为了计算$c^{\langle t \rangle}$,它会与$\tilde{c}^{\langle t \rangle}$相乘
#### 更新单元
为了要更新主题，我们需要创建一个新的向量，我们可以将其添加到之前的单元状态中。我们使用的公式是：
 $$ \tilde{c}^{\langle t \rangle} = \tanh(W_c[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_c)\tag{3} $$
最后，单元的新状态是：
 $$ c^{\langle t \rangle} = \Gamma_f^{\langle t \rangle}* c^{\langle t-1 \rangle} + \Gamma_u^{\langle t \rangle} *\tilde{c}^{\langle t \rangle} \tag{4} $$
#### 输出门
 为了决定我们将使用哪种输出，我们将使用以下两个公式：
$$ \Gamma_o^{\langle t \rangle}=  \sigma(W_o[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_o)\tag{5}$$ 
$$ a^{\langle t \rangle} = \Gamma_o^{\langle t \rangle}* \tanh(c^{\langle t \rangle})\tag{6} $$

### LSTM单元
我们根据图4来实现一个LSTM单元，步骤如下：

- 把 $a^{\langle t-1 \rangle}$ , $x^{\langle t \rangle}$连接起来变为一个矩阵：$concat = \begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle} \end{bmatrix}$
- 计算公式1-6，我们可以使用sigmoid()（在rnn_utils内）与np.tanh()。

- 计算预测 $y^{\langle t \rangle}$我们可以使用softmax()（在rnn_utils内）。

In [4]:
def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    """
    根据图4实现一个LSTM单元的前向传播。
    
    参数：
        xt -- 在时间步“t”输入的数据，维度为(n_x, m)
        a_prev -- 上一个时间步“t-1”的隐藏状态，维度为(n_a, m)
        c_prev -- 上一个时间步“t-1”的记忆状态，维度为(n_a, m)
        parameters -- 字典类型的变量，包含了：
                        Wf -- 遗忘门的权值，维度为(n_a, n_a + n_x)
                        bf -- 遗忘门的偏置，维度为(n_a, 1)
                        Wi -- 更新门的权值，维度为(n_a, n_a + n_x)
                        bi -- 更新门的偏置，维度为(n_a, 1)
                        Wc -- 第一个“tanh”的权值，维度为(n_a, n_a + n_x)
                        bc -- 第一个“tanh”的偏置，维度为(n_a, n_a + n_x)
                        Wo -- 输出门的权值，维度为(n_a, n_a + n_x)
                        bo -- 输出门的偏置，维度为(n_a, 1)
                        Wy -- 隐藏状态与输出相关的权值，维度为(n_y, n_a)
                        by -- 隐藏状态与输出相关的偏置，维度为(n_y, 1)
    返回：
        a_next -- 下一个隐藏状态，维度为(n_a, m)
        c_next -- 下一个记忆状态，维度为(n_a, m)
        yt_pred -- 在时间步“t”的预测，维度为(n_y, m)
        cache -- 包含了反向传播所需要的参数，包含了(a_next, c_next, a_prev, c_prev, xt, parameters)
        
    注意：
        ft/it/ot表示遗忘/更新/输出门，cct表示候选值(c tilda)，c表示记忆值。
    """
    
    # 从“parameters”中获取相关值
    Wf = parameters["Wf"]
    bf = parameters["bf"]
    Wi = parameters["Wi"]
    bi = parameters["bi"]
    Wc = parameters["Wc"]
    bc = parameters["bc"]
    Wo = parameters["Wo"]
    bo = parameters["bo"]
    Wy = parameters["Wy"]
    by = parameters["by"]
    
    # 获取 xt 与 Wy 的维度信息
    n_x, m = xt.shape
    n_y, n_a = Wy.shape
    
    # 1.连接 a_prev 与 xt
    contact = np.zeros([n_a + n_x, m])
    contact[: n_a, :] = a_prev
    contact[n_a :, :] = xt
    
    # 2.根据公式计算ft、it、cct、c_next、ot、a_next
    
    ## 遗忘门，公式1
    ft = rnn_utils.sigmoid(np.dot(Wf, contact) + bf)
    
    ## 更新门，公式2
    it = rnn_utils.sigmoid(np.dot(Wi, contact) + bi)
    
    ## 更新单元，公式3
    cct = np.tanh(np.dot(Wc, contact) + bc)
    
    ## 更新单元，公式4
    #c_next = np.multiply(ft, c_prev) + np.multiply(it, cct)
    c_next = ft * c_prev + it * cct
    ## 输出门，公式5
    ot = rnn_utils.sigmoid(np.dot(Wo, contact) + bo)
    
    ## 输出门，公式6
    #a_next = np.multiply(ot, np.tan(c_next))
    a_next = ot * np.tanh(c_next)
    # 3.计算LSTM单元的预测值
    yt_pred = rnn_utils.softmax(np.dot(Wy, a_next) + by)
    
    # 保存包含了反向传播所需要的参数
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)
    
    return a_next, c_next, yt_pred, cache

### LSTM的前向传播
 我们已经实现了LSTM单元的一个时间步的前向传播，现在我们要对LSTM网络进行前向传播进行计算
 <img src="images/5.png" style="width:500;height:300px;">

In [5]:
def lstm_forward(x, a0, parameters):
    """
    根据图5来实现LSTM单元组成的的循环神经网络
    
    参数：
        x -- 所有时间步的输入数据，维度为(n_x, m, T_x)
        a0 -- 初始化隐藏状态，维度为(n_a, m)
        parameters -- python字典，包含了以下参数：
                        Wf -- 遗忘门的权值，维度为(n_a, n_a + n_x)
                        bf -- 遗忘门的偏置，维度为(n_a, 1)
                        Wi -- 更新门的权值，维度为(n_a, n_a + n_x)
                        bi -- 更新门的偏置，维度为(n_a, 1)
                        Wc -- 第一个“tanh”的权值，维度为(n_a, n_a + n_x)
                        bc -- 第一个“tanh”的偏置，维度为(n_a, n_a + n_x)
                        Wo -- 输出门的权值，维度为(n_a, n_a + n_x)
                        bo -- 输出门的偏置，维度为(n_a, 1)
                        Wy -- 隐藏状态与输出相关的权值，维度为(n_y, n_a)
                        by -- 隐藏状态与输出相关的偏置，维度为(n_y, 1)
        
    返回：
        a -- 所有时间步的隐藏状态，维度为(n_a, m, T_x)
        y -- 所有时间步的预测值，维度为(n_y, m, T_x)
        caches -- 为反向传播的保存的元组，维度为（【列表类型】cache, x)）
    """
    
    # 初始化“caches”
    caches = []
    
    # 获取 xt 与 Wy 的维度信息
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wy"].shape
    
    # 使用0来初始化“a”、“c”、“y”
    a = np.zeros([n_a, m, T_x])
    c = np.zeros([n_a, m, T_x])
    y = np.zeros([n_y, m, T_x])
    
    # 初始化“a_next”、“c_next”
    a_next = a0
    c_next = np.zeros([n_a, m])
    
    # 遍历所有的时间步
    for t in range(T_x):
        # 更新下一个隐藏状态，下一个记忆状态，计算预测值，获取cache
        a_next, c_next, yt_pred, cache = lstm_cell_forward(x[:,:,t], a_next, c_next, parameters)
        
        # 保存新的下一个隐藏状态到变量a中
        a[:, :, t] = a_next
        
        # 保存预测值到变量y中
        y[:, :, t] = yt_pred
        
        # 保存下一个单元状态到变量c中
        c[:, :, t] = c_next
        
        # 把cache添加到caches中
        caches.append(cache)
    
    # 保存反向传播需要的参数
    caches = (caches, x)
    
    return a, y, c, caches

# 字符级语言模型 - 恐龙岛
欢迎来到恐龙岛，恐龙生活于在6500万年前，现在研究人员在试着复活恐龙，而你的任务就是给恐龙命名，如果一只恐龙不喜欢它的名字，它可能会狂躁不安，所以你要谨慎选择。
<img src="images/6.jpg" style="width:250;height:300px;">
你的助手已经收集了他们能够找到的所有恐龙名字，并编入了这个[数据集](dinos.txt),为了构建字符级语言模型来生成新的名称，你的模型将学习不同的名称模式，并随机生成新的名字。希望这个算法能让你和你的团队远离恐龙的愤怒。

在这里你将学习到：

- 如何存储文本数据以便使用RNN进行处理。

- 如何合成数据，通过每次采样预测，并将其传递给下一个rnn单元。

- 如何构建字符级文本生成循环神经网络。

- 为什么梯度修剪很重要?

我们将首先加载我们在rnn_utils中提供的一些函数。具体地说，我们可以使用rnn_forward和rnn_backward等函数，这些函数与前面实现的函数相同。

In [6]:
import numpy as np
import random
import time
import cllm_utils

## 问题描述
### 数据集与预处理
 我们先来读取恐龙名称的数据集，创建一个唯一字符列表（如AZ），并计算数据集和词汇量大小。

In [1]:
# 获取名称
data = open("dinos.txt", "r").read()

# 转化为小写字符
data = data.lower()

# 转化为无序且不重复的元素列表
chars = list(set(data))

# 获取大小信息
data_size, vocab_size = len(data), len(chars)

print(chars)
print("共计有%d个字符，唯一字符有%d个"%(data_size,vocab_size))

['f', 'm', 'p', 'b', '\n', 'j', 'g', 'i', 'h', 'r', 'c', 'w', 'y', 't', 'v', 'n', 'k', 'q', 'd', 's', 'x', 'e', 'o', 'a', 'u', 'z', 'l']
共计有19909个字符，唯一字符有27个


In [2]:
data = open("dinos.txt", "r").read()
data

'Aachenosaurus\nAardonyx\nAbdallahsaurus\nAbelisaurus\nAbrictosaurus\nAbrosaurus\nAbydosaurus\nAcanthopholis\nAchelousaurus\nAcheroraptor\nAchillesaurus\nAchillobator\nAcristavus\nAcrocanthosaurus\nAcrotholus\nActiosaurus\nAdamantisaurus\nAdasaurus\nAdelolophus\nAdeopapposaurus\nAegyptosaurus\nAeolosaurus\nAepisaurus\nAepyornithomimus\nAerosteon\nAetonyxAfromimus\nAfrovenator\nAgathaumas\nAggiosaurus\nAgilisaurus\nAgnosphitys\nAgrosaurus\nAgujaceratops\nAgustinia\nAhshislepelta\nAirakoraptor\nAjancingenia\nAjkaceratops\nAlamosaurus\nAlaskacephale\nAlbalophosaurus\nAlbertaceratops\nAlbertadromeus\nAlbertavenator\nAlbertonykus\nAlbertosaurus\nAlbinykus\nAlbisaurus\nAlcovasaurus\nAlectrosaurus\nAletopelta\nAlgoasaurus\nAlioramus\nAliwalia\nAllosaurus\nAlmas\nAlnashetri\nAlocodon\nAltirhinus\nAltispinax\nAlvarezsaurus\nAlwalkeria\nAlxasaurus\nAmargasaurus\nAmargastegos\nAmargatitanis\nAmazonsaurus\nAmmosaurus\nAmpelosaurus\nAmphicoelias\nAmphicoelicaudia\nAmphisaurus\nAmtocephale\nAmtosaur

这些字符是a-z（26个英文字符）加上“\n”（换行字符），在这里换行字符起到了在视频中类似的EOS（句子结尾）的作用，这里表示了名字的结束而不是句子的结尾。下面我们将创建一个字典，每个字符映射到0-26的索引，然后再创建一个字典，它将该字典将每个索引映射回相应的字符字符，它会帮助我们找出softmax层的概率分布输出中的字符。我们来创建char_to_ix 与 ix_to_char字典。

In [8]:
char_to_ix = {ch:i for i, ch in enumerate(sorted(chars))}
ix_to_char = {i:ch for i, ch in enumerate(sorted(chars))}

print(char_to_ix)
print(ix_to_char)

{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


### 模型回顾
模型的结构如下：

- 初始化参数

- 循环：

  - 前向传播计算损失

  - 反向传播计算关于损失的梯度

  - 修剪梯度以免梯度爆炸

  - 用梯度下降更新规则更新参数。

- 返回学习后了的参数
<img src="6.png" style="width:450;height:300px;">
在每个时间步，RNN会预测给定字符的下一个字符是什么。数据集  $X = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ 是一个列表类型的字符训练集，同时$Y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$在每个时间步t亦是如此，因此$y^{\langle t \rangle} = x^{\langle t+1 \rangle}$. 

## 构建模型中的模块
在这部分，我们将来构建整个模型中的两个重要的模块：
- 梯度修剪：避免梯度爆炸
- 取样:一种用来产生字符的技术
### 梯度修剪
 在这里，我们将实现在优化循环中调用的clip函数。回想一下，整个循环结构通常包括前向传播、成本计算、反向传播和参数更新。在更新参数之前，我们将在需要时执行梯度修剪，以确保我们的梯度不是“爆炸”的。

 接下来我们将实现一个修剪函数，该函数输入一个梯度字典输出一个已经修剪过了的梯度。有很多的方法来修剪梯度，我们在这里使用一个比较简单的方法。梯度向量的每一个元素都被限制在[−N，N]的范围，通俗的说，有一个maxValue（比如10），如果梯度的任何值大于10，那么它将被设置为10，如果梯度的任何值小于-10，那么它将被设置为-10，如果它在-10与10之间，那么它将不变。
 <img src="7.png" style="width:400;height:150px;">

In [9]:
def clip(gradients, maxValue):
    """
    使用maxValue来修剪梯度
    
    参数：
        gradients -- 字典类型，包含了以下参数："dWaa", "dWax", "dWya", "db", "dby"
        maxValue -- 阈值，把梯度值限制在[-maxValue, maxValue]内
        
    返回：
        gradients -- 修剪后的梯度
    """
    # 获取参数
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
    
    # 梯度修剪
    for gradient in [dWaa, dWax, dWya, db, dby]:
        np.clip(gradient, -maxValue, maxValue, out=gradient)

    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

### 采样
 现在假设我们的模型已经训练过了，我们希望生成新的文本，生成的过程如下图：
 <img src="images/8.png" style="width:500;height:300px;">
  在这幅图中，我们假设模型已经经过了训练。我们在第一步传入$x^{\langle 1\rangle} = \vec{0}$，然后让网络一次对一个字符进行采样。
 现在我们来实现sample函数，它由以下四步构成：
- 步骤1：网络的第一个“虚”输入$x^{\langle 1 \rangle} = \vec{0}$（零向量），这是在生成字符之前的默认输入，同时我们设置 $a^{\langle 0 \rangle} = \vec{0}$
- 步骤2：运行一次前向传播，然后得到$a^{\langle 1 \rangle}$ and $\hat{y}^{\langle 1 \rangle}$ 公式如下:
$$ a^{\langle t+1 \rangle} = \tanh(W_{ax}  x^{\langle t \rangle } + W_{aa} a^{\langle t \rangle } + b)\tag{1}$$

$$ z^{\langle t + 1 \rangle } = W_{ya}  a^{\langle t + 1 \rangle } + b_y \tag{2}$$

$$ \hat{y}^{\langle t+1 \rangle } = softmax(z^{\langle t + 1 \rangle })\tag{3}$$需要注意的是$\hat{y}^{\langle t+1 \rangle }$ 是一个softmax概率向量（其下的值在0到1之间，总和为1），$\hat{y}^{\langle t+1 \rangle}_i$ 表示索引“i”的字符是下一个字符的概率
- 步骤3：采样：根据 $\hat{y}^{\langle t+1 \rangle }$指定的概率分布选择下一个字符的索引，假如$\hat{y}^{\langle t+1 \rangle }_i = 0.16$那么选择索引“i”的概率为16%，为了实现它，我们可以使用np.random.choice函数，下面是np.random.choice()使用的例子：
```python
np.random.seed(0)
p = np.array([0.1, 0.0, 0.7, 0.2])
index = np.random.choice([0, 1, 2, 3], p = p.ravel())
```
这意味着你将根据分布选择索引
$P(index = 0) = 0.1, P(index = 1) = 0.0, P(index = 2) = 0.7, P(index = 3) = 0.2$.
- 步骤4：在sample()中实现的最后一步是用 $x^{\langle t +1\rangle }$的值覆盖变量x(当前存储$x^{\langle t \rangle }$)我们将通过创建一个与我们所选择的字符相对应的一个独热向量来表示 $x^{\langle t +1\rangle }$。然后在步骤1中向前传播 $x^{\langle t +1\rangle }$,并不断重复这个过程，直到得到一个“\n”字符，表明已经到达恐龙名称的末尾。

In [10]:
def sample(parameters, char_to_ix, seed):
    """
    根据RNN输出的概率分布序列对字符序列进行采样
    
    参数：
        parameters -- 包含了Waa, Wax, Wya, by, b的字典
        char_to_ix -- 字符映射到索引的字典
        seed -- 随机种子
        
    返回：
        indices -- 包含采样字符索引的长度为n的列表。
    """
    
    # 从parameters 中获取参数
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    # 步骤1 
    ## 创建独热向量x
    x = np.zeros((vocab_size,1))
    
    ## 使用0初始化a_prev
    a_prev = np.zeros((n_a,1))
    
    # 创建索引的空列表，这是包含要生成的字符的索引的列表。
    indices = []
    
    # IDX是检测换行符的标志，我们将其初始化为-1。
    idx = -1
    
    # 循环遍历时间步骤t。在每个时间步中，从概率分布中抽取一个字符，
    # 并将其索引附加到“indices”上，如果我们达到50个字符，
    #（我们应该不太可能有一个训练好的模型），我们将停止循环，这有助于调试并防止进入无限循环
    counter = 0
    newline_character = char_to_ix["\n"]
    
    while (idx != newline_character and counter < 50):
        # 步骤2：使用公式1、2、3进行前向传播
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b)
        z = np.dot(Wya, a) + by
        y = cllm_utils.softmax(z)
        
        # 设定随机种子
        np.random.seed(counter + seed)
        
        # 步骤3：从概率分布y中抽取词汇表中字符的索引
        idx = np.random.choice(list(range(vocab_size)), p=y.ravel())
        
        # 添加到索引中
        indices.append(idx)
        
        # 步骤4:将输入字符重写为与采样索引对应的字符。
        x = np.zeros((vocab_size,1))
        x[idx] = 1
        
        # 更新a_prev为a
        a_prev = a 
        
        # 累加器
        seed += 1
        counter +=1
    
    if(counter == 50):
        indices.append(char_to_ix["\n"])
    
    return indices

In [11]:
np.random.seed(2)
_, n_a = 20, 100
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}


indices = sample(parameters, char_to_ix, 0)
print("Sampling:")
print("list of sampled indices:", indices)
print("list of sampled characters:", [ix_to_char[i] for i in indices])

Sampling:
list of sampled indices: [12, 17, 24, 14, 13, 9, 10, 22, 24, 6, 13, 11, 12, 6, 21, 15, 21, 14, 3, 2, 1, 21, 18, 24, 7, 25, 6, 25, 18, 10, 16, 2, 3, 8, 15, 12, 11, 7, 1, 12, 10, 2, 7, 7, 11, 17, 24, 12, 3, 1, 0]
list of sampled characters: ['l', 'q', 'x', 'n', 'm', 'i', 'j', 'v', 'x', 'f', 'm', 'k', 'l', 'f', 'u', 'o', 'u', 'n', 'c', 'b', 'a', 'u', 'r', 'x', 'g', 'y', 'f', 'y', 'r', 'j', 'p', 'b', 'c', 'h', 'o', 'l', 'k', 'g', 'a', 'l', 'j', 'b', 'g', 'g', 'k', 'q', 'x', 'l', 'c', 'a', '\n']


## 构建语言模型
### 梯度下降
 在这里，我们将实现一个执行随机梯度下降的一个步骤的函数（带有梯度修剪）。我们将一次训练一个样本，所以优化算法将是随机梯度下降，这里是RNN的一个通用的优化循环的步骤：

- 前向传播计算损失

- 反向传播计算关于参数的梯度损失

- 修剪梯度

- 使用梯度下降更新参数

 我们来实现这一优化过程（单步随机梯度下降），这里我们提供了一些函数

```python
def rnn_forward(X, Y, a_prev, parameters):
    """
    通过RNN进行前向传播，计算交叉熵损失。

    它返回损失的值以及存储在反向传播中使用的“缓存”值。
    """
    ....
    return loss, cache
    
def rnn_backward(X, Y, parameters, cache):
    """ 
    通过时间进行反向传播，计算相对于参数的梯度损失。它还返回所有隐藏的状态
    """
    ...
    return gradients, a

def update_parameters(parameters, gradients, learning_rate):
    """
    Updates parameters using the Gradient Descent Update Rule
    """
    ...
    return parameters
```

In [12]:
def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    执行训练模型的单步优化。
    
    参数：
        X -- 整数列表，其中每个整数映射到词汇表中的字符。
        Y -- 整数列表，与X完全相同，但向左移动了一个索引。
        a_prev -- 上一个隐藏状态
        parameters -- 字典，包含了以下参数：
                        Wax -- 权重矩阵乘以输入，维度为(n_a, n_x)
                        Waa -- 权重矩阵乘以隐藏状态，维度为(n_a, n_a)
                        Wya -- 隐藏状态与输出相关的权重矩阵，维度为(n_y, n_a)
                        b -- 偏置，维度为(n_a, 1)
                        by -- 隐藏状态与输出相关的权重偏置，维度为(n_y, 1)
        learning_rate -- 模型学习的速率
    
    返回：
        loss -- 损失函数的值（交叉熵损失）
        gradients -- 字典，包含了以下参数：
                        dWax -- 输入到隐藏的权值的梯度，维度为(n_a, n_x)
                        dWaa -- 隐藏到隐藏的权值的梯度，维度为(n_a, n_a)
                        dWya -- 隐藏到输出的权值的梯度，维度为(n_y, n_a)
                        db -- 偏置的梯度，维度为(n_a, 1)
                        dby -- 输出偏置向量的梯度，维度为(n_y, 1)
        a[len(X)-1] -- 最后的隐藏状态，维度为(n_a, 1)
    """
    
    # 前向传播
    loss, cache = cllm_utils.rnn_forward(X, Y, a_prev, parameters)
    
    # 反向传播
    gradients, a = cllm_utils.rnn_backward(X, Y, parameters, cache)
    
    # 梯度修剪，[-5 , 5]
    gradients = clip(gradients,5)
    
    # 更新参数
    parameters = cllm_utils.update_parameters(parameters,gradients,learning_rate)
    
    return loss, gradients, a[len(X)-1]

In [13]:
np.random.seed(1)
vocab_size, n_a = 27, 100
a_prev = np.random.randn(n_a, 1)
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}
X = [12,3,5,11,22,3]
Y = [4,14,11,22,25, 26]

loss, gradients, a_last = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
print("Loss =", loss)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("np.argmax(gradients[\"dWax\"]) =", np.argmax(gradients["dWax"]))
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])
print("a_last[4] =", a_last[4])

Loss = 126.50397572165345
gradients["dWaa"][1][2] = 0.19470931534725341
np.argmax(gradients["dWax"]) = 93
gradients["dWya"][1][2] = -0.007773876032004315
gradients["db"][4] = [-0.06809825]
gradients["dby"][1] = [0.01538192]
a_last[4] = [-1.]


### 训练模型
给定恐龙名称的数据集，我们使用数据集的每一行(一个名称)作为一个训练样本。每100步随机梯度下降，你将抽样10个随机选择的名字，看看算法是怎么做的。记住要打乱数据集，以便随机梯度下降以随机顺序访问样本。当examples[index]包含一个恐龙名称（String）时，为了创建一个样本（X,Y），你可以使用这个
```python
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix["\n"]]
```
需要注意的是我们使用了`index= j % len(examples)`，其中`j = 1....num_iterations`，为了确保examples[index]总是有效的（index小于len(examples)）,rnn_forward()会将X的第一个值None解释为$x^{\langle 0 \rangle} = \vec{0}$。此外，为了确保Y等于X，会向左移动一步，并添加一个附加的“\n”以表示恐龙名称的结束。

In [14]:
def model(data, ix_to_char, char_to_ix, num_iterations=3500, 
          n_a=50, dino_names=7,vocab_size=27):
    """
    训练模型并生成恐龙名字
    
    参数：
        data -- 语料库
        ix_to_char -- 索引映射字符字典
        char_to_ix -- 字符映射索引字典
        num_iterations -- 迭代次数
        n_a -- RNN单元数量
        dino_names -- 每次迭代中采样的数量
        vocab_size -- 在文本中的唯一字符的数量
    
    返回：
        parameters -- 学习后了的参数
    """
    
    # 从vocab_size中获取n_x、n_y
    n_x, n_y = vocab_size, vocab_size
    
    # 初始化参数
    parameters = cllm_utils.initialize_parameters(n_a, n_x, n_y)
    
    # 初始化损失
    loss = cllm_utils.get_initial_loss(vocab_size, dino_names)
    
    # 构建恐龙名称列表
    with open("dinos.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    # 打乱全部的恐龙名称
    np.random.seed(0)
    np.random.shuffle(examples)
    
    # 初始化LSTM隐藏状态
    a_prev = np.zeros((n_a,1))
    
    # 循环
    for j in range(num_iterations):
        # 定义一个训练样本
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix["\n"]]
        
        # 执行单步优化：前向传播 -> 反向传播 -> 梯度修剪 -> 更新参数
        # 选择学习率为0.01
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters)
        
        # 使用延迟来保持损失平滑,这是为了加速训练。
        loss = cllm_utils.smooth(loss, curr_loss)
        
        # 每2000次迭代，通过sample()生成“\n”字符，检查模型是否学习正确
        if j % 2000 == 0:
            print("第" + str(j+1) + "次迭代，损失值为：" + str(loss))
            
            seed = 0
            for name in range(dino_names):
                # 采样
                sampled_indices = sample(parameters, char_to_ix, seed)
                cllm_utils.print_sample(sampled_indices, ix_to_char)
                
                # 为了得到相同的效果，随机种子+1
                seed += 1
            
            print("\n")
    return parameters

In [15]:
#开始时间
start_time = time.clock()

#开始训练
parameters = model(data, ix_to_char, char_to_ix, num_iterations=3500)

#结束时间
end_time = time.clock()

#计算时差
minium = end_time - start_time

print("执行了：" + str(int(minium / 60)) + "分" + str(int(minium%60)) + "秒")

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


第1次迭代，损失值为：23.087336085484605
Nkzxwtdmfqoeyhsqwasjkjvu
Kneb
Kzxwtdmfqoeyhsqwasjkjvu
Neb
Zxwtdmfqoeyhsqwasjkjvu
Eb
Xwtdmfqoeyhsqwasjkjvu


第2001次迭代，损失值为：27.884160491415773
Liusskeomnolxeros
Hmdaairus
Hytroligoraurus
Lecalosapaus
Xusicikoraurus
Abalpsamantisaurus
Tpraneronxeros


执行了：0分4秒


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [16]:
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
import numpy as np
import IPython
import sys
from music21 import *
from grammar import *
from qa import *
from preprocess import * 
from music_utils import *
from data_utils import *

Using TensorFlow backend.


ModuleNotFoundError: No module named 'music21'